In [ ]:
#!git clone git@github.com:micheltokic/crawlingrobot.git
#!pip install -e crawlingrobot/

In [7]:
import sys
import numpy as np
import pygame
import os
#os.environ['SDL_VIDEODRIVER']='dummy'
import gym
import gym_crawlingrobot

# 1) Manual

In [8]:
def manual_robot_control (env):
    
    done = False
    action = None
    env.reset()
    cum_reward = 0
    step = 0

    while True:
        # process pygame event loop
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    pygame.quit()
                    return
                elif event.key == pygame.K_UP or event.key == pygame.K_w:
                    env.step(0)
                    action = 0
                elif event.key == pygame.K_RIGHT or event.key == pygame.K_d:
                    action = 1
                elif event.key == pygame.K_DOWN or event.key == pygame.K_s:
                    action = 2
                elif event.key == pygame.K_LEFT or event.key == pygame.K_a:
                    action = 3
                elif event.key == pygame.K_r:
                    env.reset()
                    action = 3
                elif event.key == pygame.K_SPACE:
                    env.robot.render_intermediate_steps = not env.robot.render_intermediate_steps

                if action:
                    obs, reward, done, info = env.step(action)
                    action = None
                    cum_reward += reward
                    step += 1
                    print (f"step={step}, action={action}, reward={reward:.2f}, cum_reward={cum_reward:.2f}, done={done}")
                if done:
                    env.reset()
                    action = 3
                    cum_reward = 0
                    step = 0

            env.render()


In [ ]:
env = gym.make('crawlingrobot-discrete-v1', rotation_angles=5, goal_distance=700, window_size=(640, 480), render_intermediate_steps=True, plot_steps_per_episode=True)
env.robot.mode = 2 # => Use WASD or Arrow Keys to control the robot's arms
manual_robot_control (env)
pygame.quit()

# 2) Q-Learning with discrete actions

In [9]:
# function maps the 2D observation (x, y) to a single state number n 
def obs_to_number(obs, obs_max):
    return int(obs[0] * obs_max + obs[1])

def q_agent(Q, obs_max, env, learn=True, render=False, alpha=1, gamma=0.95, epsilon=0.2, maxSteps=10000, episodes=200):
    
    print (f"Q.shape={Q.shape}")
    np.set_printoptions(threshold=sys.maxsize)

    for episode in range (episodes):
        done = False
        init_obs = env.reset()
        state = obs_to_number(init_obs, obs_max)
        step = 0
        cum_reward =0 

        while not done and step < maxSteps:

            # action selection
            if np.random.rand() < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(Q[state])

            # perform action in environment
            nextObs, reward, done, _ = env.step(action)
            nextState = obs_to_number(nextObs, obs_max)
            cum_reward += reward

            # environment rendering
            if render:
                env.render()

                # process pygame event loop
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        return
                    elif event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_ESCAPE:
                            pygame.quit()
                            return
                        if event.key == pygame.K_SPACE:
                            env.robot.render_intermediate_steps = not env.robot.render_intermediate_steps

            # Q-learning
            if learn:
                Q[state, action] += alpha * (reward + gamma * np.max(Q[nextState]) - Q[state, action])

            # time transition
            state = nextState
            step += 1
            
        res = 0
        if len(env.robot.episode_time_results) > 0:
            res = env.robot.episode_time_results[-1]
        print(f"episode={episode} took {step} steps => cumulative reward: {cum_reward:.2f}")
        
    pygame.quit()
    return

#########################
# Initialize Q function
#########################
obs_max = env.observation_space.high[0] + 1  # currently 5
Q = np.zeros([obs_max ** len(env.observation_space.high), env.action_space.n])

## 2.1) Learn Q function

In [10]:
pygame.quit()

# instantiate environment
env = gym.make('crawlingrobot-discrete-v1', rotation_angles=5, goal_distance=700)

# learn Q function
q_agent(Q=Q, obs_max=obs_max, env=env, gamma=0.9, epsilon=0.1, episodes=100, render=False, learn=True)

Q.shape=(25, 4)
episode=0 took 642 steps => cumulative reward: 510.92
episode=1 took 102 steps => cumulative reward: 526.93
episode=2 took 106 steps => cumulative reward: 521.42
episode=3 took 97 steps => cumulative reward: 524.49
episode=4 took 99 steps => cumulative reward: 524.60
episode=5 took 91 steps => cumulative reward: 517.62
episode=6 took 106 steps => cumulative reward: 526.31
episode=7 took 96 steps => cumulative reward: 516.85
episode=8 took 102 steps => cumulative reward: 510.40
episode=9 took 101 steps => cumulative reward: 520.16
episode=10 took 98 steps => cumulative reward: 510.63
episode=11 took 90 steps => cumulative reward: 529.07
episode=12 took 95 steps => cumulative reward: 519.99
episode=13 took 93 steps => cumulative reward: 529.75
episode=14 took 99 steps => cumulative reward: 525.11
episode=15 took 90 steps => cumulative reward: 527.26
episode=16 took 86 steps => cumulative reward: 526.94
episode=17 took 84 steps => cumulative reward: 527.95
episode=18 took 

## 2.2) Evaluate policy derived from Q function

In [15]:
# evalue Q function
pygame.quit()
env = gym.make('crawlingrobot-discrete-v1', rotation_angles=5, goal_distance=700, window_size=(640, 480), plot_steps_per_episode=True)
q_agent(Q=Q, obs_max=obs_max, env=env, episodes=20, epsilon=0.1, render=True, learn=False)

Q.shape=(25, 4)
episode=0 took 83 steps => cumulative reward: 525.37
episode=1 took 76 steps => cumulative reward: 518.50
episode=2 took 73 steps => cumulative reward: 510.60
episode=3 took 90 steps => cumulative reward: 514.26
episode=4 took 75 steps => cumulative reward: 513.55
episode=5 took 72 steps => cumulative reward: 525.98
episode=6 took 79 steps => cumulative reward: 519.48
episode=7 took 82 steps => cumulative reward: 521.05
episode=8 took 99 steps => cumulative reward: 516.18
episode=9 took 83 steps => cumulative reward: 527.85
episode=10 took 76 steps => cumulative reward: 513.33
episode=11 took 78 steps => cumulative reward: 513.62
episode=12 took 82 steps => cumulative reward: 526.64
episode=13 took 74 steps => cumulative reward: 513.35
episode=14 took 79 steps => cumulative reward: 519.14
episode=15 took 75 steps => cumulative reward: 510.83
episode=16 took 86 steps => cumulative reward: 516.32
episode=17 took 79 steps => cumulative reward: 521.67
episode=18 took 80 ste

# 3) PPO2 control with continuous actions

In [ ]:
#!pip install stable-baselines
#!pip install tensorflow==1.15

In [ ]:
import pygame
import sys
import os
import numpy as np
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines.bench import Monitor
from stable_baselines import PPO2
import gym
import gym_crawlingrobot

In [ ]:
# define callback class for event loop cleanup
from stable_baselines.common.callbacks import BaseCallback

class PyGameEventLoopCallback(BaseCallback):
    
    render = False
    
    def __init__(self, verbose=0, render=False):
        super(PyGameEventLoopCallback, self).__init__(verbose)
        self.render = render
        # Those variables will be accessible in the callback
        # (they are defined in the base class)
        # The RL model
        # self.model = None  # type: BaseRLModel
        # An alias for self.model.get_env(), the environment used for training
        # self.training_env = None  # type: Union[gym.Env, VecEnv, None]
        # Number of time the callback was called
        # self.n_calls = 0  # type: int
        # self.num_timesteps = 0  # type: int
        # local and global variables
        # self.locals = None  # type: Dict[str, Any]
        # self.globals = None  # type: Dict[str, Any]
        # The logger object, used to report things in the terminal
        # self.logger = None  # type: logger.Logger
        # # Sometimes, for event callback, it is useful
        # # to have access to the parent object
        # self.parent = None  # type: Optional[BaseCallback]
    
    def _on_step(self) -> bool:
        
        robot_env = self.training_env.venv.envs[0]
        
        # process pygame event loop
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    pygame.quit()
                    return
                if event.key == pygame.K_SPACE:
                    robot_env.robot.render_intermediate_steps = not robot_env.robot.render_intermediate_steps

        if self.render:
            robot_env.render()
        
        return bool

In [ ]:
log_dir = ""
ppo2 = "ppo2"
os.makedirs(ppo2, exist_ok=True)

def ppo2_learn(env, render=False):
    env = VecNormalize(DummyVecEnv([lambda: Monitor(env, log_dir)]), norm_obs=True, norm_reward=True)
    model = PPO2(MlpPolicy, env, verbose=1, learning_rate=0.15)
    model.learn(total_timesteps=30000, callback=PyGameEventLoopCallback(render=render))
    
    #model.learn(total_timesteps=30000)
    model.save("ppo2/ppo2_crawling_robot")
    env.save("ppo2/vec_normalize.pkl")
 
    del model, env


def ppo2_run_policy(env, render=False, episodes=1):
    env = DummyVecEnv([lambda: Monitor(env, log_dir)])
    env = VecNormalize.load("ppo2/vec_normalize.pkl", env)
    model = PPO2.load("ppo2/ppo2_crawling_robot")

    # visualization callback
    cb = PyGameEventLoopCallback(render=render)
    cb.training_env = env

    env.training = False
    
    for e in range (episodes): 

        obs = env.reset()
        done = False
        cum_reward = 0
        step = 0

        while not done:
            action, _states = model.predict(obs)
            obs, _reward, done, info = env.step(action)
            reward = env.get_original_reward() # returns the last unnormalized reward
            cb._on_step()
            cum_reward += reward[0]
            step += 1
            print (f"episode={e}, step={step}, action={action}, reward={reward[0]:.2f}, cum_reward={cum_reward:.2f}, done={done}")


### Train policy for 30000 timesteps (no GUI)

In [ ]:
robot_env = gym.make('crawlingrobot-continuous-v1', goal_distance=2500)
ppo2_learn(env=robot_env, render=False)
pygame.quit()

### Evaluate policy (with GUI)

In [ ]:
#robot_env_nogui = gym.make('crawlingrobot-continuous-v1', goal_distance=2500, plot_steps_per_episode=False, render_intermediate_steps=False)
robot_env_gui = gym.make('crawlingrobot-continuous-v1', goal_distance=700, window_size=(640, 480), plot_steps_per_episode=True, render_intermediate_steps=True)
ppo2_run_policy(env=robot_env_gui, episodes=30, render=True)
pygame.quit()